In [16]:

import logging
import multiprocessing
import os
from collections import Counter
from typing import List, Union

from gensim.models import FastText
from indicnlp.tokenize.indic_tokenize import trivial_tokenize_indic
from tqdm import tqdm

tqdm.pandas()

In [17]:
# set up the logging to monitor gensim
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt='%H:%M:%S',
    level=logging.INFO,
)

logger = logging.getLogger(__name__)

## Utilities

In [18]:
def tokenize_text(text: List[str]) -> List[List[str]]:
    """Tokenize text"""
    return [trivial_tokenize_indic(sent) for sent in tqdm(text, desc='tokenize', unit=' sentences')]

In [23]:
def train_fasttext(tokenized_text: List[List[str]], size: int = 100, window: int = 5, min_count: int = 1,
                   epochs: int = 10,
                   random_seed: int = 123, vec_file_path: Union[str, None] = None, ):
    "Learn fasttext embeddings"
    # count the number of cores
    cores = multiprocessing.cpu_count()
    # create fasttext model
    model = FastText(
        size=size,
        window=window,
        min_count=min_count,
        workers=cores - 1,
        seed=random_seed,
    )
    # build vocab
    model.build_vocab(sentences=tokenized_text, progress_per=1000000)  # show progress after processing every 1M words
    # train
    model.train(sentences=tokenized_text, total_examples=model.corpus_count, epochs=epochs,
                report_delay=10)  # show progress after every 10 seconds
    if vec_file_path is not None:
        model.wv.save_word2vec_format(vec_file_path, binary=False)
    return model

## Load data
For learning the Odia word embeddings, we need monolingual Odia text data. You can possibly scrape data from an online source such as Wikipedia. For our experiments now, let's take the Odia monolingual text data available as part of the Indic NLP corpus.

In [21]:
filename = os.path.join('data/or')
assert os.path.isfile(filename)  # sanity check

In [22]:
with open(filename, 'r', encoding='utf-8') as f:
    lines = [s.strip() for s in tqdm(f.readlines(), desc='read lines from file')]

read lines from file: 100%|██████████| 3594672/3594672 [00:02<00:00, 1568395.63it/s]


In [24]:
# tokenize
tokens = tokenize_text(lines)

tokenize: 100%|██████████| 3594672/3594672 [01:24<00:00, 42326.60 sentences/s]


In [25]:
num_running_toks, num_unique_toks = 0, 0
counter = Counter()
for toks in tqdm(tokens, desc='compute frequencies of tokens', unit=' sentences'):
    counter.update(toks)

compute frequencies of tokens: 100%|██████████| 3594672/3594672 [00:20<00:00, 173846.92 sentences/s]


In [26]:
print(f'Number of sentences: {len(lines):,}')
print(f'Number of unique words or equivalantly, the size of vocabulary: {len(counter):,}')
print(f'Number of running words: {sum([freq for _, freq in counter.items()]):,}')

Number of sentences: 3,594,672
Number of unique words or equivalantly, the size of vocabulary: 778,862
Number of running words: 51,151,273


In [27]:

# most common words
counter.most_common(n=20)

[('।', 3393061),
 (',', 1191253),
 ('ଓ', 534792),
 ('ଏହି', 437185),
 ('ପାଇଁ', 373726),
 ('ସେ', 240775),
 ('ବୋଲି', 239837),
 ('ପରେ', 224959),
 ('କରି', 221628),
 ('ଏକ', 213516),
 ('ମଧ୍ୟ', 210907),
 ('ଏବଂ', 198988),
 ('କରିଥିଲେ', 195168),
 ('ସହ', 177040),
 ('-', 174796),
 ('ଖବର', 169373),
 ('.', 166728),
 ('କରିବା', 166276),
 ('ନେଇ', 161728),
 ('ବେଳେ', 156327)]

## Learn embeddings

In [28]:
fasttext_model = train_fasttext(tokenized_text=tokens, size=100, window=5, min_count=20, epochs=10, random_seed=123,
                                vec_file_path=os.path.join('embeddings.txt'))

INFO - 21:05:54: resetting layer weights
INFO - 21:06:07: collecting all words and their counts
INFO - 21:06:07: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:06:12: PROGRESS: at sentence #1000000, processed 14406915 words, keeping 356423 word types
INFO - 21:06:16: PROGRESS: at sentence #2000000, processed 28227997 words, keeping 518060 word types
INFO - 21:06:21: PROGRESS: at sentence #3000000, processed 42532970 words, keeping 692676 word types
INFO - 21:06:23: collected 778862 word types from a corpus of 51151273 raw words and 3594672 sentences
INFO - 21:06:23: Loading a fresh vocabulary
INFO - 21:06:25: effective_min_count=20 retains 72827 unique words (9% of original 778862, drops 706035)
INFO - 21:06:25: effective_min_count=20 leaves 49262024 word corpus (96% of original 51151273, drops 1889249)
INFO - 21:06:26: deleting the raw counts dictionary of 778862 items
INFO - 21:06:26: sample=0.001 downsamples 22 most-common words
INFO - 21:06:26: downsamp

## Evaluate embeddings
Here we evaluate the embeddings learned by just 👀 at the neighbours of a few words and examining if they are similar.



In [37]:
## find words similar to a given word
fasttext_model.wv.most_similar('ଗଛ', topn=10)  # TODO: misspell it

[('ଗଛଡାଳ', 0.8625774383544922),
 ('ବରଗଛ', 0.8207435607910156),
 ('ଗଛର', 0.812586784362793),
 ('ଗଛଟି', 0.7994915246963501),
 ('ଆମ୍ବଗଛ', 0.7820898294448853),
 ('ଫୁଲଗଛ', 0.750383734703064),
 ('କଦଳୀଗଛ', 0.7475508451461792),
 ('ଶାଳଗଛ', 0.7465659379959106),
 ('ଗଛଟିଏ', 0.7456701993942261),
 ('ବାଉଁଶ', 0.7355741262435913)]

In [38]:
fasttext_model.wv.most_similar('ସଙ୍ଗୀତ', topn=10)

[('ସଂଗୀତ', 0.9623903036117554),
 ('ସଂଙ୍ଗୀତ', 0.9553436040878296),
 ('ସଙ୍ଗୀତର', 0.9166804552078247),
 ('ସଙ୍ଗୀତକାର', 0.8910577893257141),
 ('ସଂଗୀତର', 0.8831936120986938),
 ('ସଂଗୀତକାର', 0.8764158487319946),
 ('ସଙ୍ଗୀତଜ୍ଞ', 0.8699268102645874),
 ('ଗୀତିନାଟ୍ୟ', 0.8680843710899353),
 ('ନୃତ୍ୟଗୀତ', 0.8561346530914307),
 ('ନାଟ୍ୟ', 0.8338338136672974)]

In [39]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ୍ର', topn=10)

[('ଚଳଚିତ୍ର', 0.9308711290359497),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.9191179275512695),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.9073216915130615),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.8922181129455566),
 ('ସିନେମା', 0.840882420539856),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.8325262069702148),
 ('ଚଳଚ୍ଚିତ୍ରରେ', 0.8220148682594299),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.8215944766998291),
 ('ଫିଲ୍ମ', 0.8206902742385864),
 ('ସିନେ', 0.8191198110580444)]

**Try some misspelled words**

In [40]:
fasttext_model.wv.most_similar('ଚଳଚ୍ଚିତ', topn=10)

[('ଚଳଚ୍ଚିତ୍ର', 0.8137073516845703),
 ('ଚଳଚ୍ଚିତ୍ରଟିର', 0.7877084612846375),
 ('ଚଳଚ୍ଚିତ୍ରଟି', 0.7712979316711426),
 ('ସମୁଚ୍ଚିତ', 0.7514731884002686),
 ('ଚଳଚ୍ଚିତ୍ରର', 0.7456707954406738),
 ('ଚଳଚ୍ଚିତ୍ରଟିରେ', 0.7123273611068726),
 ('ସିନେ', 0.7101430892944336),
 ('ସିନେମାଟୋଗ୍ରାଫର', 0.709763765335083),
 ('ଲୋକପ୍ରିୟ', 0.7081252336502075),
 ('ଫିଲ୍ମ', 0.7038418650627136)]

In [41]:
fasttext_model.wv.most_similar('ସଗୀତ', topn=10)

[('ଗୀତ', 0.9316926002502441),
 ('ସଂଙ୍ଗୀତ', 0.891626238822937),
 ('ନାଚଗୀତ', 0.8895268440246582),
 ('ନୃତ୍ୟଗୀତ', 0.8842304944992065),
 ('ସଂଗୀତ', 0.883371114730835),
 ('ସଙ୍ଗୀତ', 0.8755859136581421),
 ('ଗୀତ୍', 0.8577066659927368),
 ('ଗୀତିନାଟ୍ୟ', 0.8234955072402954),
 ('ସଂଗୀତର', 0.8086755871772766),
 ('ସଙ୍ଗୀତର', 0.8027265071868896)]